<a href="https://colab.research.google.com/github/Stephanie9606/Multimodal_Deep_Learning/blob/main/multiclass_bert_memes_TF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
!pip3 install bert-for-tf2
!pip3 install sentencepiece

     |████████████████████████████████| 41 kB 183 kB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-py3-none-any.whl size=30535 sha256=e49cd8875d9ab652294078c562fe58034b1a67f1472e14be3d3aac2978668626
  Stored in directory: /root/.cache/pip/wheels/47/b6/e5/8c76ec779f54bc5c2f1b57d2200bb9c77616da83873e8acb53
  Created wheel for params-flow: filename=params_flow-0.8.2-py3-none-any.whl size=19472 sha256=a0d72c725cfef389b2b97cec229c326b5e68cf78e6edbbf439ccfc926dcb23a5
  Stored in directory: /root/.cache/pip/wheels/0e/fc/d2/a44fff33af0f233d7def6e7de413006d57c10e10ad736fe8f5
  Created wheel for py-params: filename=py_params-0.10.2-py3-none-any.whl size=7911 sha256=2572a11532eece8a5f20bde25f9d8222cf160978022e9f936133e56ace6b9c0c
  Stored in directory: /root/.cache/pip/wheels/e1/11/67/33cc51bbee127cb8fb2ba549cd29109b2f22da43ddf9969716
Successfully built bert-for-tf2 params-flow py-params
     |████████████████████████████████| 1.2 MB 7.1 MB/s 


In [3]:
import tensorflow_hub as hub
from tensorflow.keras.models import Model
from bert.tokenization.bert_tokenization import FullTokenizer
from tensorflow.keras.layers import Dense, Input, Dropout, Flatten, LSTM

In [4]:
# Set up the device for GPU usage
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [5]:
# connect to google drive for files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Pre-processing

In [6]:
memesDF = pd.read_csv('/content/drive/MyDrive/top5_memes_tidy.tsv', sep='\t')

In [7]:
memesDF = memesDF.iloc[:, 2:9]

In [8]:
memesDF['AltText'] = memesDF['AltText'].astype(pd.StringDtype())
memesDF['MemeLabel'] = memesDF['MemeLabel'].astype(pd.StringDtype())
memesDF['CaptionText'] = memesDF['CaptionText'].astype(pd.StringDtype())
memesDF['HashId'] = memesDF['HashId'].astype(pd.StringDtype())
memesDF['ImageURL'] = memesDF['ImageURL'].astype(pd.StringDtype())
memesDF['ImagePath'] = memesDF['ImagePath'].astype(pd.StringDtype())
memesDF['Image'] = memesDF['Image'].astype(pd.StringDtype())

In [9]:
memesDF.dtypes

AltText        string
CaptionText    string
ImageURL       string
HashId         string
MemeLabel      string
ImagePath      string
Image          string
dtype: object

In [10]:
memesDF.nunique()

AltText        5899
CaptionText    5887
ImageURL       5944
HashId         5944
MemeLabel         5
ImagePath      5944
Image          5944
dtype: int64

In [11]:
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

le = preprocessing.LabelEncoder()
le.fit(memesDF.MemeLabel)
memes5cDF = memesDF
memes5cDF['MemeLabel'] = le.transform(memesDF.MemeLabel)

In [12]:
# train test split
from sklearn.model_selection import train_test_split

me_trainDF, me_testDF = train_test_split(memes5cDF, test_size=0.2, random_state=15)

In [13]:
y_train = to_categorical(me_trainDF.MemeLabel)
y_test = to_categorical(me_testDF.MemeLabel)

In [14]:
me_trainDF.head(3)

,AltText,CaptionText,ImageURL,HashId,MemeLabel,ImagePath,Image
3853,Uncle Sam | WHEN UNCLE SAM SEES DA BOOTY | ima...,WHEN UNCLE SAM SEES DA BOOTY,http://i.imgflip.com/25o9xw.jpg,1da120833e5c8326aa67e1134fac76485e7586b0725e3f...,3,/content/memes_image/25o9xw.jpg,25o9xw.jpg
5093,Peter Parker Cry | DO YOU HAVE DIABITIES? YAH?...,DO YOU HAVE DIABITIES? YAH? HAHAHA,http://i.imgflip.com/25o7bj.jpg,db6deae5b06b5337658752fe796e29f294d6adfa6a0084...,0,/content/memes_image/25o7bj.jpg,25o7bj.jpg
73,Who Killed Hannibal | SOME GUY DEAD GUY SOME G...,SOME GUY DEAD GUY SOME GUY THAT SHOT A DEAD GUY,http://i.imgflip.com/3s89iu.jpg,85c26a7db0688f29d7f5a8b7e0c0ab42cbfa4c03a9283b...,4,/content/memes_image/3s89iu.jpg,3s89iu.jpg


# Load Pre-trained Model

In [15]:
!pip install transformers
import transformers

     |████████████████████████████████| 4.0 MB 8.4 MB/s 
     |████████████████████████████████| 596 kB 11.4 MB/s 
     |████████████████████████████████| 895 kB 64.4 MB/s 
     |████████████████████████████████| 6.6 MB 57.4 MB/s 
     |████████████████████████████████| 77 kB 8.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [16]:
from transformers import AutoTokenizer,TFBertModel
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
bert = TFBertModel.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/502M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [17]:
# Tokenize the input (takes some time) 
# here tokenizer using from bert-base-cased
x_train = tokenizer(
    text=me_trainDF.CaptionText.tolist(),
    add_special_tokens=True,
    max_length=128,
    truncation=True,
    padding="max_length", 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True
    #verbose = True
    )
x_test = tokenizer(
    text=me_testDF.CaptionText.tolist(),
    add_special_tokens=True,
    max_length=128,
    truncation=True,
    padding="max_length", 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True
    #verbose = True
    )

In [18]:
input_ids = x_train['input_ids']
attention_mask = x_train['attention_mask']

In [19]:
max_len = 128
input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
embeddings = bert(input_ids,attention_mask = input_mask)[0] 
out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
#out = Dense(128, activation='relu')(out)
#out = tf.keras.layers.Dropout(0.1)(out)
#out = Dense(32,activation = 'relu')(out)
y = Dense(5,activation = 'softmax')(out)
model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
model.layers[2].trainable = True

In [22]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 128)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 128,                                           

In [20]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [21]:
train_history = model.fit(
    x ={'input_ids':x_train['input_ids'],'attention_mask':x_train['attention_mask']} ,
    y = y_train,
    validation_data = (
    {'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']}, y_test
    ),
    epochs=10,
    batch_size=32
)

Epoch 1/10
149/149 [==============================] - 145s 846ms/step - loss: 0.8379 - accuracy: 0.6946 - val_loss: 0.5755 - val_accuracy: 0.7965
Epoch 2/10
149/149 [==============================] - 129s 864ms/step - loss: 0.5120 - accuracy: 0.8238 - val_loss: 0.4903 - val_accuracy: 0.8284
Epoch 3/10
149/149 [==============================] - 131s 883ms/step - loss: 0.3618 - accuracy: 0.8753 - val_loss: 0.5346 - val_accuracy: 0.8234
Epoch 4/10
149/149 [==============================] - 133s 893ms/step - loss: 0.2219 - accuracy: 0.9281 - val_loss: 0.5155 - val_accuracy: 0.8259
Epoch 5/10
149/149 [==============================] - 133s 892ms/step - loss: 0.1443 - accuracy: 0.9542 - val_loss: 0.5974 - val_accuracy: 0.8209
Epoch 6/10
149/149 [==============================] - 133s 895ms/step - loss: 0.0841 - accuracy: 0.9760 - val_loss: 0.6099 - val_accuracy: 0.8251
Epoch 7/10
149/149 [==============================] - 133s 892ms/step - loss: 0.0789 - accuracy: 0.9720 - val_loss: 0.6268 -